# Parsing HAProxy Logs

This notebook demonstrates real world log parsing examples. For this notebook we will use the popular HAProxy
format.

This notebook will cover
+ Streaming log entry event level logs from Kafka using Streamz
 - If using the [custreamz Docker image](https://github.com/rapidsai/notebooks-contrib/tree/master/intermediate_notebooks/examples/custreamz/BUILD.md) Kafka is installed and running for you by default.
 - If not using the provided Docker image users can refer to: https://kafka.apache.org/quickstart for reference on how to start a local Kafka cluster.
+ Counting log word occurrences - (Word Count)
+ Calculating average backend response time

Ok so what do these HAProxy logs look like? Well here is an example.

```{"logline": "[haproxy@10.0.0.1] <134>May 29 19:08:36 haproxy[113498]: 45.26.605.15:38738 [29/May/2019:19:08:36.691] HTTPS:443~ HTTP_ProvisionManagers/mp3 4/5/0/1/1 200 6182 - - --NI 3/3/0/0/0 0/0 {|} "GET /v2/serverinfo HTTP/1.1"}```

It is unlikely you have a Kafka topic with these log messages in it already so lets generate some sample messages for you. First we need to install a few required dependencies and define some global configurations however.

Users using the [custreamz Docker image](https://github.com/rapidsai/notebooks-contrib/tree/master/intermediate_notebooks/examples/custreamz/BUILD.md) do not need to perform the following conda install since the container already has the needed dependencies. Running it again will not causes any problems but is not necessary.


In [1]:
!conda install -c conda-forge -c anaconda -y streamz python-confluent-kafka networkx graphviz

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



# Global Configurations

In [2]:
import confluent_kafka as ck
from confluent_kafka.admin import AdminClient, NewTopic

num_messages_to_produce = 500000

kafka_brokers = ['localhost:9092'] # This is a list of your Kafka brokers and ports.
topic = 'haproxy-logs'

kafka_conf = {'bootstrap.servers': kafka_brokers[0], 'group.id': 'custreamz', 'session.timeout.ms': 60000}  # Kafka configuration parameters. Any additional Kafka configurations can be placed here ...

producer = ck.Producer(kafka_conf)  # Kafka producer

# !!! - COMMENT OUT THE BELOW IF YOU WOULD NOT LIKE TO HAVE THIS SCRIPT CREATE A KAFKA TOPIC FOR YOU !!!
admin = AdminClient({'bootstrap.servers': kafka_brokers[0]})
fs = admin.create_topics([NewTopic(topic, 1, 1)], request_timeout=15.0)

for topic, f in fs.items():
    try:
        f.result()
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topci {}: {}".format(topic, e))


Failed to create topci haproxy-logs: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'haproxy-logs' already exists."}


# Generate Sample HAProxy Data
First thing is first. Lets generate some sample HAProxy logs into our Kafka environment so the following examples have something to pull from. This code has nothing to do with RAPIDS but rather just a simple script for generating sample HAProxy logs and publishing them into Kafka.

This is certainly not the most efficient way to write to Kafka but it is the most simple for example purposes. Please be patient while the produce occurs.

In [3]:
import json
from random import randrange

sample_data = {}
sample_data['log_ip'] = ['10.0.0.3', '14.2.3.4', '15.2.6.9', '10.2.34.6', '10.23.34.1']
sample_data['syslog_timestamp'] = ['May 28 2019 00:00:09', 'May 28 2019 00:00:10', 'May 28 2019 00:00:11',\
                                   'May 28 2019 00:00:39','May 28 2019 00:00:51', 'May 28 2019 00:10:09']
sample_data['program'] = ['haproxy']
sample_data['pid'] = [113345, 756487, 352453, 352465, 164541]
sample_data['client_ip'] = ['156.23.224.56', '126.52.74.15', '247.81.56.21', '26.245.255.1', '255.116.145.2']
sample_data['client_port'] = [13345, 56487, 52453, 52465, 64541]
sample_data['accept_date'] = ['28/May/2019:00:10:09.492', '28/May/2019:00:09:10.006', '28/May/2019:00:02:10.748',\
                              '28/May/2019:00:20:10.891', '28/May/2019:00:02:10.461', '28/May/2019:00:02:11.959']
sample_data['frontend_name'] = ['px-http', 'https:443', 'tx-http']
sample_data['server_name'] = ['srv1', 'srv2', 'srv3', 'srv4', 'srv5']
sample_data['time_request'] = [0, 1, 2, 3]
sample_data['time_queue'] = [0, 1, 2, 3]
sample_data['time_backend_connect'] = [1, 2, 3]
sample_data['time_backend_response'] = [2, 3, 4, 5, 6, 7, 8, 9]
sample_data['time_duration'] = [13, 14, 16, 20, 23, 25]
sample_data['http_status_code'] = [200, 400, 201, 401, 403]
sample_data['bytes_read'] = [4, 573, 442, 234, 124, 1567]
sample_data['captured_request'] = ['-']
sample_data['captured_response'] = ['-']
sample_data['termination_state'] = ['----', 'PH--', 'CR--', '--NI', '--SG']
sample_data['actconn'] = [1, 2, 3, 4]
sample_data['feconn'] = [2, 3, 5, 7, 8]
sample_data['beconn'] = [0, 1, 2, 3, 4]
sample_data['srvconn'] = [0, 1, 3]
sample_data['retries'] = [0, 1, 2]
sample_data['srv_queue'] = [0, 1, 2, 3]
sample_data['backend_queue'] = [0, 2, 3, 4, 5, 7, 8, 9]

cols = ['log_ip','syslog_timestamp','program','pid','client_ip','client_port',\
        'accept_date','frontend_name','backend_name','server_name','time_request',\
        'time_queue','time_backend_connect', 'time_backend_response', 'time_duration',\
        'http_status_code', 'bytes_read', 'captured_request', 'captured_response',\
        'termination_state','actconn','feconn','beconn','srvconn','retries','srv_queue','backend_queue']

def generate_log():
    log_skelton = "[haproxy@{0}] <134>{1} {2}[{3}]: {4}:{5} [{6}] {7} {8}/{9} {10}/{11}/{12}/{13}/{14} {15} {16} {17} {18} {19} {20}/{21}/{22}/{23}/{24} {25}/{26}"
    values = []
    for idx, col in enumerate(cols):
        if col in sample_data:
            value_list = sample_data[col]
            values.append(value_list[randrange(len(value_list))])
        else:
            values.append(values[-1])
    dict_out = {}    
    dict_out["logline"] = log_skelton.format(*values)
    return json.dumps(dict_out)


count = 0
print("Generating: {} sample HAProxy logs to Kafka topic: {} at broker: {}".format(num_messages_to_produce, topic, kafka_brokers[0]))
try:
    while count < num_messages_to_produce:
        producer.produce(topic, generate_log())
        
        # Default max local buffer size is 100,000 so flush after 90,000
        if count % 90000 == 0:
            print(count)
            producer.flush()
        count = count + 1
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
    
producer.flush()
print("{} - HAProxy logs written to Kafka".format(count))

Generating: 500000 sample HAProxy logs to Kafka topic: haproxy-logs at broker: localhost:9092
0
90000
180000
270000
360000
450000
500000 - HAProxy logs written to Kafka


# Counting Word Occurrences

A common task associated with log monitoring is counting the occurrence of a "word" with in a line of text. Streamz makes it easy to do this and we will show below how to create a map function to examine the HAProxy logs and count the number of "403" "http_status_code" responses.

If a user needs to examine raw text they can also make use of nvstrings (now custrings, the GPU-accelerated string manipulation library) to tokenise all the messages in the batch.

In [4]:
import cudf
from streamz import Stream
from streamz.dataframe import DataFrame
import json

# Create a Dask LocalCUDACluster
import dask
from dask_cuda import LocalCUDACluster
from distributed import Client
cluster = LocalCUDACluster()
client = Client(cluster)

# Piece meal parsing of the HAProxy logs to cuDF. Ugly but gets the job done.
def haproxy_lines_to_cudf(messages):
    
    # Take the batch of logs from kafka and combine them into line delimted string for GPU parsing
    json_input_string = "\n".join([msg.decode('utf-8') for msg in messages])
    gdf = cudf.read_json(json_input_string, lines=True, engine='cudf')
    
    clean_df = gdf['logline'].str.split(' ')

    clean_df['log_ip'] = clean_df[0].str.lstrip('[haproxy@').str.rstrip(']')
    clean_df.drop_column(0)

    clean_df[1] = clean_df[1].str.split('>')[1]
    syslog_timestamp = clean_df[1].data.cat([clean_df[2].data, clean_df[3].data, clean_df[4].data], sep=' ')
    clean_df['syslog_timestamp'] = cudf.Series(syslog_timestamp)
    for col in [1,2,3,4]:
        clean_df.drop_column(col)

    program_pid_df = clean_df[5].str.split('[')
    program_sr = program_pid_df[0]
    pid_sr = program_pid_df[1]
    clean_df['program'] = program_sr
    clean_df['pid'] = pid_sr.str.rstrip(']:')
    clean_df = clean_df.drop(labels=[5])
    del program_pid_df

    client_pid_port_df = clean_df[6].str.split(':')
    clean_df['client_ip'], clean_df['client_port'] = client_pid_port_df[0], client_pid_port_df[1]
    clean_df.drop_column(6)
    del client_pid_port_df

    clean_df['accept_date'] = clean_df[7].str.lstrip('[').str.rstrip(']')
    clean_df.drop_column(7)

    clean_df.rename({8: 'frontend_name'}, inplace=True)
    backend_server_df = clean_df[9].str.split('/')
    clean_df['backend_name'], clean_df['server_name'] = backend_server_df[0], backend_server_df[1]
    clean_df.drop_column(9)

    time_cols = ['time_request', 'time_queue', 'time_backend_connect', 'time_backend_response', 'time_duration']
    time_df = clean_df[10].str.split('/')
    for col_id, col_name in enumerate(time_cols):
        clean_df[col_name] = time_df[col_id]
    clean_df.drop_column(10)
    del time_df

    clean_df.rename({11: 'http_status_code'}, inplace=True)
    clean_df.rename({12: 'bytes_read'}, inplace=True)
    clean_df.rename({13: 'captured_request', 14: 'captured_response', 15: 'termination_state'}, inplace=True)

    con_cols = ['actconn', 'feconn', 'beconn', 'srvconn', 'retries']
    con_df = clean_df[16].str.split('/')
    for col_id, col_name in enumerate(con_cols):
        clean_df[col_name] = con_df[col_id]
    clean_df.drop_column(16)
    del con_df

    q_df = clean_df[17].str.split('/')
    clean_df['srv_queue'], clean_df['backend_queue'] = q_df[0], q_df[1]
    clean_df.drop_column(17)
    del q_df
    
    clean_df['time_backend_response'] = clean_df['time_backend_response'].astype('int')
    clean_df['http_status_code'] = clean_df['http_status_code'].astype('int')
    
    # Return the cuDF dataframe so that other operations in the stream can use it
    return clean_df
    
# We now use Streamz to create a Stream from Kafka by polling the topic every 1s.
source = Stream.from_kafka_batched(topic, kafka_conf, npartitions=1, poll_interval='1s', asynchronous=True, dask=True)

# When started this returns to us a RAPIDS cuDF
sdf = source.map(haproxy_lines_to_cudf)

The above snippet defines a function for parsing a batch of HAProxy logs into a cuDF and also defining streamz code that will be used to invoke that function. Keep in mind the Stream is not running until we later invoke "start" so right now we are still just building up our configuration.

In [5]:
# Counts the number of 403 from the "http_status_code"
def count_auth_errors(gdf):
    auth_errors = gdf[gdf['http_status_code'] == 403].http_status_code.count()
    return auth_errors

# Count the number of 200 codes from "http_status_code". Of course it would be better to make a common function for this but being verbose for demonstration purposes.
def count_ok_codes(gdf):
    ok_codes = gdf[gdf['http_status_code'] == 200].http_status_code.count()
    return ok_codes

# Here we demonstrate splitting from the parent "sdf" stream source and branch out to different streams.
auth_codes = sdf.map(count_auth_errors).gather().sink(lambda err_count: print("{:,} - 403 codes were encountered.".format(err_count)))
ok_codes = sdf.map(count_ok_codes).gather().sink(lambda ok_count: print("{:,} - 200 codes were encountered.".format(ok_count)))

# Calculating Average Backend Response Time

Our code above has already taken care of creating a [Streamz Streaming DataFrame](https://streamz.readthedocs.io/en/latest/dataframes.html) using RAPIDS cuDF for us. So no need to do that again. What we can do is reuse that object and inquire different questions on it with another stream without having to pull all of the messages from Kafka again (slowest part of pipeline). Lets take a look at that.

In [6]:
def calculate_avg_response(gdf):
    return gdf['time_backend_response'].mean()

auth_resps = sdf.map(calculate_avg_response).gather().sink(lambda avg_time: print("{:.2f} second(s) average backend response time".format(avg_time)))

Now that we have finished defining our Stream we need to actually start the stream so that it will begin to pull messages from Kafka. Take note that since streamz is an async platform you might think the job has exited and left you with no output. This should return in a couple of seconds however. Lets start the stream!

In [7]:
source.start()

5.50 second(s) average backend response time
99,853 - 200 codes were encountered.
99,667 - 403 codes were encountered.


# Cleanup - Optional

If you would like to remove the Kafka Topic used in this example then you can run the below cell.

In [8]:
import confluent_kafka as ck
from confluent_kafka.admin import AdminClient, NewTopic

admin = AdminClient({'bootstrap.servers': kafka_brokers[0]})
fs = admin.delete_topics([topic], operation_timeout=30)

for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} deleted".format(topic))
    except Exception as e:
        print("Failed to delete topic {}: {}".format(topic, e))

Topic haproxy-logs deleted
